<a href="https://colab.research.google.com/github/BDH-teacher/RL_from_basics/blob/main/RL_from_basic_ch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Method

## 라이브러리 import

In [1]:
import random
import numpy as np

from tqdm import trange

## Grid World 클래스

In [2]:
class GridWorld():
    def __init__(self):
        self.x=0
        self.y=0

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()

        reward = -1 # 보상은 항상 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_right(self):
        self.y += 1
        if self.y > 3:
            self.y = 3

    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0

    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0

    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3

    def is_done(self):
        if self.x == 3 and self.y == 3:
            return True
        else :
            return False

    def get_state(self):
        return (self.x, self.y)

    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

- 환경에 해당하는 GridWorld 클래스 정의
   - Step 함수 : 에이전트로부터 액션을 받아서 상태 변이를 일으키고, 보상을 정해주는 함수
   - is_done 함수 : 에피소드가 끝났는지 판별해주는 함수, 에이전트가 종료 상태에 도달했으면 True, 아니면 False를 리턴
   - reset함수 : 에이전트가 종료 상태에 도달했을 때 처음상태로 돌려 놓는 함수

## Agent 클래스

In [3]:
class Agent():
    def __init__(self):
        pass

    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

- 에이전트에 해당하는 클래스
   - 4방향 uniform 랜덤 액션을 선택
   - select_action 함수 : 각각 1/4확률로 4가지 액션 중 하나를 선택하는 함수

## 메인 함수

In [4]:
# 초기 셋팅

env = GridWorld()
agent = Agent()
data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] # 테이블 초기화
gamma = 1.0
reward = -1
alpha = 0.001

In [5]:
for k in trange(50000): # 총 5만 번의 에피소드 진행
  done = False
  history = []

  # 에피소드 1회 진행
  while not done:
    action = agent.select_action()
    (x,y), reward, done = env.step(action)
    history.append((x,y,reward))

  # env 초기화
  env.reset()

  # 에피소드 종료 후 테이블 업데이트
  cum_reward = 0

  for transition in history[::-1]: # history의 뒤쪽부터 차례차례 리턴을 계산
    x, y, reward = transition
    data[x][y] = data[x][y] + alpha * (cum_reward-data[x][y])
    cum_reward = reward + gamma * cum_reward

100%|██████████| 50000/50000 [00:06<00:00, 7791.68it/s] 


In [6]:
# 학습이 끝난 후 데이터를 출력해보기 위한 코드

for row in data:
    print(row)

[-62.57203295125681, -60.45246296319531, -53.73226841910136, -49.76126757346607]
[-61.18306140378648, -58.9247882581445, -52.95649761610383, -47.69758443289825]
[-60.628457532890565, -57.405001984758, -43.866333383549865, -31.38582596324584]
[-59.42589290505005, -53.677216795074536, -33.39860927505324, 0.0]


- 실제로 학습을 진행하는 메인 함수
  - GrideWorld 클래스의 인스턴스인 env라는 변수 선언
  - data 변수 선언 : 테이블에 각 상태의 가치를 임의의 값으로 초기화
    - MC는 data를 업데이트하는 방식으로 진행됨
  - gamma : 1, alpha : 0.001

---

<br/>

- while loop는 랜덤 에이전트가 경험을 쌓는 과정이고, for loop는 쌓은 경험을 이용해 테이블을 업데이트함
  - for loop안의 cum_reward는 리턴을 의미함

<br/>

- 리턴($G_t$)의 정의 <br/>
$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 R_{t+4} + \cdots$ <br/>
$= R_{t+1} + \gamma \left(R_{t+2} + \gamma R_{t+3} + \gamma^2 R_{t+4} + \cdots\right)$ <br/>
$= R_{t+1} + \gamma G_{t+1}$

   - 리턴 $G_{t+1}$과 $G_t$사이의 재귀적인 관계가 존재함
   - cum_reward라는 변수에 $G_{t+1}$가 담겨 있기에 $\gamma$가 곱해지고 $R_{t+1}$이 더해지며 $G_t$가 됨

# Temporal Difference

## 라이브러리 import 부터 Agent 클래스 까지
- 앞의 MC와 동일

## 메인 함수

In [7]:
    #TD
    env = GridWorld()
    agent = Agent()
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    gamma = 1.0
    reward = -1
    alpha = 0.01 # MC에 비해 큰 값을 사용

In [8]:
for k in trange(50000): # 총 5만번의 에피소드 진행
  done = False
  while not done:
    x, y = env.get_state()
    action = agent.select_action()
    (x_prime, y_prime), reward, done = env.step(action)
    x_prime, y_prime = env.get_state()

    # 한 번의 step이 진행되자 마자 바로 테이블의 데이터를 업데이트 해줌
    data[x][y] = data[x][y] + alpha*(reward+gamma*data[x_prime][y_prime]-data[x][y])
  env.reset()

100%|██████████| 50000/50000 [00:03<00:00, 13840.56it/s]


In [9]:
for row in data:
  print(row)

[-59.23433158596225, -57.357720164197566, -53.97985606061329, -52.233553391839656]
[-57.710650172122165, -54.36239719063453, -50.05296181821229, -45.93090100650767]
[-54.88291811398165, -50.79613818411838, -41.902543046020654, -33.109063269480195]
[-52.25350398897838, -46.28072535115985, -32.52640497786485, 0]


- MC에 비해 학습 코드가 간결함
  - 한 번의 액션마다 데이터 테이블이 업데이트됨
  - TD가 MC에 비해 학습의 변동성이 작은 덕분에 그만큼 큰 폭의 업데이트가 가능하기에 alpha의 값을 키울 수 있음